In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from scnn.convertors import F2S, S2F
from scnn.bipolar_functions.operations import *

In [2]:
x_in = torch.rand(500,1,10) * 2 -1
y_in = torch.rand(500, 10,1) * 2 -1
(x_in@y_in).squeeze().shape

torch.Size([500])

In [50]:
def get_mae(seq_len, input_dim, scalar):
    
    f2s = F2S(seq_len)
    s2f = S2F()
    x_in = (torch.rand(1,input_dim) * 2 -1)/(input_dim**0.5)
    y_in = (torch.rand(1,input_dim) * 2 -1)/(input_dim**0.5)
    print((x_in * y_in).sum(-1))
    x_in_s = f2s(x_in)
    y_in_s = f2s(y_in)

    ground = torch.tanh(scalar*(x_in * y_in).sum(-1)) #shape = [500]
    print(ground)
    
    mode = 'mux'
    if mode == 'mux':
        out = []
        for i in range(1):
            dot = scaled_dot(x_in_s[i],y_in_s[i])
            out.append(s2f(stanh(dot,int(2*input_dim*scalar))))
        mae_1 = torch.abs(torch.stack(out) -ground).mean()
        # return mae

    mode = 'apc'
    if mode == 'apc':
        inputs = mul(x_in_s,y_in_s)
        
        r = get_btanh_states(input_dim, scalar)
        print('btanh states',r)
        out = s2f(apc_btanh(inputs, r=r, num_au_layers=0))
        print(out)
        mae_2 = torch.abs(out-ground).mean()

        

    mode = 'axpc'
    if mode == 'axpc':
        out = s2f(apc_btanh(inputs, r=r, num_au_layers=1))
        mae_3 = torch.abs(out-ground).mean()
        # 
    # print('mux',mae_1)
    # print('apc',mae_2)
    # print('axpc',mae_3)
    return mae_1, mae_2, mae_3

In [51]:
get_mae(seq_len=1024, input_dim=100,scalar=2)

tensor([0.0388])
tensor([0.0774])
btanh states 420
tensor([0.2090])


(tensor(0.5871), tensor(0.1316), tensor(0.1492))

In [48]:
np.tanh(0.5991*2)

0.8331047465040887